# __HyDE-Q Pipeline__

In [1]:
import os
import time
import pickle
import random
import faiss
import warnings
warnings.filterwarnings('ignore') 

import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
from nltk import sent_tokenize
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

from hyde.HyDE import HyDEQ
from hyde.Promptor import Promptor
from hyde.Searcher import FaissSearcher
from hyde.Generator import Generator

#### **① 데이터셋 로드**

In [2]:
# train set과 validation set의 context를 모두 사용
dataset = load_dataset('squad_kor_v1')
contexts = dataset['train']['context'] + dataset['validation']['context']
contexts = list(set(contexts))

In [3]:
len(contexts)

10563

#### **② HyDE-Q Pipe Line 선언**

In [4]:
# 검색모델(unsupervised)과 생성모델 로드
encoder = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
model = AutoModelForCausalLM.from_pretrained('beomi/KoAlpaca-Polyglot-5.8B')
tokenizer = AutoTokenizer.from_pretrained('beomi/KoAlpaca-Polyglot-5.8B')

tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(30003, 4096)

In [5]:
# 프롬프터, 생성모델, 임베딩 모델, 검색도구를 선언
promptor = Promptor('koquad')
generator = Generator(model, tokenizer)
searcher = FaissSearcher(encoder, contexts, 'koquad_sbert.index')

Load Faiss from disc


In [6]:
# HyDEQ의 Pipe Line
hydeq = HyDEQ(promptor, generator, encoder, searcher.index)

#### **③ 질의로부터 가짜 답안(Hypothesis) 생성**

In [7]:
# 쿼리와 프롬프트 확인
query = '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?'
prompt = hydeq.prompt(query)
prompt

'다음 질문에 대답할 수 있는 텍스트를 만들어주세요. ###질문: 전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은? ###답변:'

In [8]:
hypothesis_documents = hydeq.generate_hypothesis(query, n=5)
for idx, doc in enumerate(hypothesis_documents):
    print(f'HyDE Generated Document: {idx}')
    print(doc.strip())

HyDE Generated Document: 0
미국의 근본주의 기독교인들은 여전히 노아가 하나님의 계획으로 이 세상에 태어났으며, 어린 시절 그가 창조하였던 동물들의 집합체(친구들)로부터 인류가 언젠가는 새로운 세상으로 발전할 수 있을 것이라고 믿고 있다.
HyDE Generated Document: 1
말라카 문화와 히브리어 문화, 그리고 히브리어 성경에 대한 오해와 잘못된 이해를 기반으로 하는 이주해온 아브람의 자손 중 가증한 물건이 첩의 품속에서 발견되어 노아의 방주를 역사적 사실로 인정하게 되었습니다.
HyDE Generated Document: 2
존재하지 않습니다. 대신, 현재의 신학계에서는 노아의 방주가 역사적 사실이 아니며, 신화적 창작물로 인식하는 경향이 있습니다. 대부분의 현대 신학들은 노아의 방주를 문자 그대로 적용하지 않고, 비유적 해석을 통하여 다른 의미를 도출해냅니다.
HyDE Generated Document: 3
성서에 근거하여 말씀드리자면 노아의 방주를 역사적 사실로 인식하는 이들은 일신론자들이 됩니다.
HyDE Generated Document: 4
미국의 정통파 신학계(American Orthodoxical Society)입니다.


#### **④ 가짜 답안을 이용해 유사 문서 검색**

In [9]:
# 가짜 답안을 인코딩
hyde_vector = hydeq.encode_documents(hypothesis_documents)
print(hyde_vector.shape)

(1, 768)


In [10]:
# 결과 확인
hits = hydeq.search(hyde_vector, k=100)
hits = [{'index': idx, 'score': score} for score, idx in zip(hits[0][0], hits[1][0])]
hits = sorted(hits, key=lambda x: x['score'], reverse=True)

hits_lst = []
for idx, hit in enumerate(hits):
  if idx < 5:
      print(f'HyDE-Q Retrieved Document: {idx}')
      print("\t{:.3f}\t{}".format(hit['score'], searcher.all_contexts[hit['index']]))
  hits_lst.append(searcher.all_contexts[hit['index']])

HyDE-Q Retrieved Document: 0
	8.057	역사학과 과학의 발달이 더뎠던 고대사회에서는, 성경이 단순한 교리적인 부분 뿐 아니라 역사책으로서의 권위도 높았기에 노아의 방주를 역사적인 존재로서 다루고 있었다. 이는 제칠일안식교에서 비롯된 의사과학의 한 종류인 유사지질학인 홍수지질학과 같은 것에 영향을 주었으며, 과거 신학에서는 이러한 근본주의적 해석을 받아들여 역사와 사회적인 모든 부분에 있어 성경을 교과서로 채택할 것을 촉구했다. 이러한 홍수지질학을 주장했던 유사지질학자들은 성경에 나오는 노아의 홍수가 어딘가에 그 흔적이 남아 있을것이라고 주장하며 노아의 방주를 찾기 위한 노력을 했다고 주장한다. 이들은 같은 메소포타미아 지방의 신화인 이슬람교 경전이나 길가메쉬 서사시등의 신화를 들어서 이를 근거라고 주장하기도 했다. 그러나 이러한 전통적 근본주의적 시각은 과거에는 상당히 힘을 얻었으나, 역사학과 과학의 발달에 따라 힘을 잃게 되었고, 홍수지질학은 유사과학으로서 남게 되었다. 현대에는 뒤의 실존논란에서 다루는 것처럼 이러한 근본주의적 해석은 비과학적인 해석으로 여기는 것이 일반적이지만, 남침례교로 대표되는 극보수주의계열 기독교에서는 아직도 이것이 받아들여지고 있다.
HyDE-Q Retrieved Document: 1
	7.790	역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러

#### **⑤ 쿼리를 이용해 Re-Rank**

In [11]:
# line을 기준으로 re-rank (정답은 1번째 문서)
rerank_hits = hydeq.rerank_by_line(hits_lst, query)
rerank_hits_lst = []
for idx, hit in enumerate(rerank_hits):
  if idx < 5:
      print(f'HyDEQ Re-ranked Retrieved Document: {id}')
      print("\t{:.3f}\t{}".format(hit['score'], hit['text']))
  rerank_hits_lst.append(hit['text'])

HyDEQ Re-ranked Retrieved Document: <built-in function id>
	0.716	역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다
HyDEQ Re-ranked Retrieved Document: <built-in function id>
	0.648	물론 노아의 방주가 신학과 신앙에서 중요한 영향을 차지하는 것은 사실이나, 현재 노아의 방주가 역사적으로 실존한다는 주장은 그 증거가 존재하지 않기에 관련 학계로부터 전혀 인정받지 못하고 있으며 그 실존과 안정성에 대한 수많은 논란이 있다. 한국창조과학회 등에서는 제칠일안식교를 기반으로 한 홍수지질학적 주장들을을 내어 놓고 있지만, 사실과 다른 근거들을 바탕으로 주장하므로 신뢰하기 힘든 것들이 전부라 할 수 있다. 그러므로 현재 노아의 방주가 실존한다는 주장은 그 증거가 존재하지 않기에 관련 학계로부터 전혀 인정받지 못하고 있다. 모든 과학관련 학계에서는 노아의 방주의 구조나 재질등이 실제로 존재할 수 없는 설화속 이야기라는 데에 동의하고 있다.
HyDEQ Re-ranked Retrieved Document: <built-in function id>
	0.638	프리스틀리는 워링턴 거주 시절에는 다른 일 때문에 신학 연구에 몰두하지 못

#### **⑥ 가장 유사도가 큰 문서를 참조해 정답 생성**

In [20]:
# 쿼리와 프롬프트
query = '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?'

reference = ''
for idx in range(0, 1):
    reference += f'{rerank_hits[idx]["text"]}'

prompt = promptor.build_prompt(query, reference)
prompt

'다음 텍스트를 참고해서 답해주세요. ###질문:전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은? ###텍스트:역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다 ###답변:'

In [22]:
prediction = hydeq.generate_answer(query, reference, n=1)
print(prediction[0])

전통 신학계에서는 노아의 방주를 역사적 사실로 인정하려 했고, 이에 따라 노아가 창조의 선(善)하고 의로운 요셉의 아들, 그리고 예수님의 어머니 마리아와 함께 에 등장하였다 라는 이야기가 완성되었다. 하지만 과학적으로는 증명되지 않은 내용이기 때문에, 현재는 사용되지 않는 것이 일반적이다. 그러나 전통적인 시각에서는 여전히 노아가 선지자였다는 것, 그리고 하느님의 뜻을 충족시키기 위해 인간적인 모습으로 지상에 나타났다는 것을 중요하게 생각한다. 노아의 방주는 현재는 상징적인 의미로 사용되는 것이 보통이다. 누구나 가지고 있는 과거의 신앙, 그리고 그 바탕이 된 문화적 유산은 지속되어야 한다는 것이다. 


#### **⑥ 검색과 생성의 정확도 확인**

In [23]:
# 각 샘플에 대해 정답지를 만들어 종합해 채점
re_answer = '역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다'
ge_answer = '보수적 근본주의계열의 개신교'

def create_answer_sheet(query, re_answer, ge_answer, prediction, hits_lst):
    answer_sheet = {
        'top_1': 'O' if re_answer in hits_lst[:1] else 'X',
        'top_5': 'O' if re_answer in hits_lst[:5] else 'X',
        'top_10': 'O' if re_answer in hits_lst[:10] else 'X',
        'top_30': 'O' if re_answer in hits_lst[:30] else 'X',
        'top_50': 'O' if re_answer in hits_lst[:50] else 'X',
        'query': query,
        're_answer': re_answer,
        'ge_answer': ge_answer,
        'predict': prediction
    }
    return answer_sheet

create_answer_sheet(query, re_answer, ge_answer, prediction, rerank_hits_lst)

{'top_1': 'O',
 'top_5': 'O',
 'top_10': 'O',
 'top_30': 'O',
 'top_50': 'O',
 'query': '전통 신학계의 근본주의적 시작을 여전히 받아들여 노아의 방주를 역사적 사실로 인식하는 집단은?',
 're_answer': '역사학과 과학이 발달하지 않았던 과거 전통 신학계에서는 근본주의적 시각을 받아들여 노아의 방주를 역사적 사실로 기술하려 했으며, 이러한 관점은 아직도 과학과 역사학에 어두운 보수적 근본주의계열의 개신교에서만 받아들여지고 있다. 하지만 역사학과 과학의 발달로 인해, 노아의 방주의 실존에 대한 의문이 제기가 되고, 세계적 홍수가 존재할 수 없음이 밝혀짐에 따라 현대 신학계에서는 비록 노아의 홍수가 과학적으로 실존하지는 않았지만 그 자체의 의미는 신학적으로 매우 중요하며, 이에 대한 해석은 다양하게 이루어지고 있으며, 대부분의 기독교(가톨릭, 개신교를 포함한 대부분)에서는 노아의 방주는 상징적 의미로 받아들여진다. 그러므로 과학과는 상관없이 신학적으로 노아의 방주 자체의 의미는 중요하게 해석된다고 한다',
 'ge_answer': '보수적 근본주의계열의 개신교',
 'predict': ['전통 신학계에서는 노아의 방주를 역사적 사실로 인정하려 했고, 이에 따라 노아가 창조의 선(善)하고 의로운 요셉의 아들, 그리고 예수님의 어머니 마리아와 함께 에 등장하였다 라는 이야기가 완성되었다. 하지만 과학적으로는 증명되지 않은 내용이기 때문에, 현재는 사용되지 않는 것이 일반적이다. 그러나 전통적인 시각에서는 여전히 노아가 선지자였다는 것, 그리고 하느님의 뜻을 충족시키기 위해 인간적인 모습으로 지상에 나타났다는 것을 중요하게 생각한다. 노아의 방주는 현재는 상징적인 의미로 사용되는 것이 보통이다. 누구나 가지고 있는 과거의 신앙, 그리고 그 바탕이 된 문화적 유산은 지속되어야 한다는 것이다. ']}